In [1]:
import pandas as pd 
from pulp import *

In [2]:
probB=LpProblem("Work",LpMinimize)
month = [1,2,3,4,5,6]
demand = {1:100,
          2:100,
          3:115,
          4:125,
          5:140,
          6:150}

salary_train = 3300
salary_untrain = 3000
#Decision Variable:
train_var = LpVariable.dicts("a",month,lowBound=0,cat="Continuous")
train_last_month = LpVariable.dicts("b",month,lowBound=0,cat="Continuous")
trained = LpVariable.dicts("c",month,lowBound=0,cat="Continuous")
untrained = LpVariable.dicts("d",month,lowBound=0,cat="Continuous")
a_untrain = LpVariable.dicts("e",month,lowBound=0,cat="Continuous")


print(trained)
print('\n')
print(untrained)
print('\n')
print(a_untrain)
print('\n')
print(train_var)

{1: c_1, 2: c_2, 3: c_3, 4: c_4, 5: c_5, 6: c_6}


{1: d_1, 2: d_2, 3: d_3, 4: d_4, 5: d_5, 6: d_6}


{1: e_1, 2: e_2, 3: e_3, 4: e_4, 5: e_5, 6: e_6}


{1: a_1, 2: a_2, 3: a_3, 4: a_4, 5: a_5, 6: a_6}


In [3]:
#Objective Funtion:
a = sum([untrained[i]*salary_untrain+trained[i]*salary_train for i in range(1,7)])
probB+= a 

#Formulate the constraints:

#1.Training number should be smaller than availble.
for i in range(1,7):
    probB+=train_var[i]<=a_untrain[i]
#2.Production Meet The Demand
for i in range(1,7):
    probB+=trained[i]*1.2+untrained[i]>=demand[i]
#3 Non-Neg
#4 Train-Last-Month
probB += train_last_month[1]==0
for i in range(2,6):
    probB +=train_last_month[i+1]==train_var[i]*0.9
#5 Available Trained Per Month:
probB += trained[1]==0
for i in range(2,7):# i from 2 to 6 
    sum_trained = 0
    for j in range(1,i+1):
        sum_trained += train_last_month[j]
    probB +=trained[i]==sum_trained
#6
#How many untrained in each month
probB += untrained[1]==145-train_var[1]
for i in range(2,7):
    probB += untrained[i]==untrained[i-1]*0.9-train_var[i]
#7
#How many are available to train in the beginning.
probB += a_untrain[1]==145
for i in range(2,7):
    probB += a_untrain[i] == untrained[i-1]*0.9+train_var[i-1]*0.1

In [5]:
probB.solve()
print("Total Cost = ",value(probB.objective))
for v in probB.variables():
    print(v.varValue)

Total Cost =  2126363.7486
45.0
47.919918
32.545504
4.7939128
0.0
0.0
0.0
48.266599
43.127927
29.290953
4.3145216
0.0
0.0
48.266599
91.394525
120.68548
125.0
125.0
100.0
42.080082
5.3265698
-0.0
-0.0
0.0
145.0
94.5
42.664065
8.0484632
0.47939128
0.0
